In [34]:
import pandas as pd
import yfinance as yf
import numpy as np
import plotly.graph_objects as go
from PIL import Image
import warnings
import pickle
import io
import os


warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [35]:
def construct_chart(data):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data['Open'],
        high=data['High'],
        low=data['Low'],
        close=data['Close']
    ))
    fig.update_layout(
        width=1000, 
        height=700
    )

    fig.update_layout(
        plot_bgcolor='black',  
        paper_bgcolor='black',  
        xaxis=dict(
            showgrid=False,      
            zeroline=False,      
            showticklabels=False 
        ),
        yaxis=dict(
            showgrid=False,      
            zeroline=False,      
            showticklabels=False 
        ),
        font=dict(color='white')  
    )
    img_bytes=fig.to_image(format='jpg')
    img=Image.open(io.BytesIO(img_bytes))
    return img


def img_preprocess(img):
    img = img.resize((224,224))
    img= np.array(img)
    img=img[30:len(img)-55]

    img = Image.fromarray(img)
    img = img.resize((224,224))
    img= (np.array(img))
    for k in range(len(img)):
        for j in range(len(img[k])):
            pic1=(img[k,j]==(np.array([255,255,255])))
            if pic1[0]!=False and pic1[1]!=False and pic1[2]!=False:
                img[k,j]=np.array([0,0,0])
    img=img/255.0
    return img

In [36]:
def check_hummer_candle(df):
    df.reset_index(inplace=True,drop=True)
    candle_volume=df['High'].iloc[1]- df['Low'].iloc[1]
    if df['Close'].iloc[1]>=df['Open'].iloc[1]:
        top_wedge=df['High'].iloc[1]-df['Close'].iloc[1]
    else:
        top_wedge=df['High'].iloc[1]-df['Open'].iloc[1]
    if top_wedge/candle_volume>=0.5:
        return True
    else:
        return False


def check_morstar_candle(df):
    df.reset_index(inplace=True,drop=True)
    candle_volume=df['High'].iloc[1]- df['Low'].iloc[1]
    if df['Close'].iloc[1]>=df['Open'].iloc[1]:
        bottom_wedge=df['Open'].iloc[1]-df['Low'].iloc[1]
    else:
        bottom_wedge=df['Close'].iloc[1]-df['Low'].iloc[1]
    if bottom_wedge/candle_volume>=0.5:
        return True
    else:
        return False


def check_engulbull_candle(df):
    if df['Close'].iloc[1]>= df['Open'].iloc[1] and df['Close'].iloc[0]<= df['Open'].iloc[0]:
        if df['Open'].iloc[1]<=df['Close'].iloc[0] and df['Close'].iloc[1]>=df['High'].iloc[0]:
            return True
    return False

def check_engulbear_candle(df):
    if df['Close'].iloc[1]<= df['Open'].iloc[1] and df['Close'].iloc[0]>= df['Open'].iloc[0]:
        if df['Open'].iloc[1]>=df['Close'].iloc[0] and df['Close'].iloc[1]<=df['Low'].iloc[0]:
            return True
    return False


In [ ]:
# I used the BTC-USD pair, but you can use any pair or stock in any timeframe 
# (the smaller the timeframe, the more data you get).

daily_data=yf.download("BTC-USD",interval='1d', start='2020-10-10')
for i in range(0,len(daily_data)-1):
    img=construct_chart(daily_data[i:i+2])
    chart=img_preprocess(img)
    data_list=[]
    if check_morstar_candle(daily_data[i:i+2]):
        label=np.array([1,0,0,0,0])
        data_tuple=(chart,label)
    elif check_hummer_candle(daily_data[i:i+2]):
        label=np.array([0,1,0,0,0])
        data_tuple=(chart,label)
    elif check_engulbull_candle(daily_data[i:i+2]):
        label=np.array([0,0,1,0,0])
        data_tuple=(chart,label)
    elif check_engulbear_candle(daily_data[i:i+2]):
        label=np.array([0,0,0,1,0])
        data_tuple=(chart,label)
    else:
        label=np.array([0,0,0,0,1])
        data_tuple=(chart,label)
    data_list.append(data_tuple)

    # Save (image,label) in data folder  
    with open(f'data//data{i}.pkl', 'wb') as f:
        pickle.dump(data_list, f)